In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
nvda = yf.Ticker('NVDA')

nvda_hist = nvda.history(period='5y', interval='1d', end='2025-06-02')

print(nvda_hist)

                                 Open        High         Low       Close  \
Date                                                                        
2020-06-22 00:00:00-04:00    9.269447    9.499938    9.242785    9.495452   
2020-06-23 00:00:00-04:00    9.519621    9.610821    9.376593    9.418953   
2020-06-24 00:00:00-04:00    9.445120    9.525107    9.114459    9.205161   
2020-06-25 00:00:00-04:00    9.325014    9.473773    9.152084    9.458822   
2020-06-26 00:00:00-04:00    9.468791    9.468791    9.095023    9.124924   
...                               ...         ...         ...         ...   
2025-06-12 00:00:00-04:00  141.970001  145.000000  141.850006  145.000000   
2025-06-13 00:00:00-04:00  142.479996  143.580002  140.850006  141.970001   
2025-06-16 00:00:00-04:00  143.350006  146.179993  143.199997  144.690002   
2025-06-17 00:00:00-04:00  144.490005  145.220001  143.779999  144.119995   
2025-06-18 00:00:00-04:00  144.009995  145.649994  143.119995  145.479996   

In [ ]:
nvda_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-06-22 00:00:00-04:00,9.269447,9.499938,9.242785,9.495452,398468000,0.0,0.0
2020-06-23 00:00:00-04:00,9.519621,9.610821,9.376593,9.418953,375108000,0.0,0.0
2020-06-24 00:00:00-04:00,9.445120,9.525107,9.114459,9.205161,449372000,0.0,0.0
2020-06-25 00:00:00-04:00,9.325014,9.473773,9.152084,9.458822,376072000,0.0,0.0
2020-06-26 00:00:00-04:00,9.468791,9.468791,9.095023,9.124924,592084000,0.0,0.0
...,...,...,...,...,...,...,...
2025-06-12 00:00:00-04:00,141.970001,145.000000,141.850006,145.000000,162365000,0.0,0.0
2025-06-13 00:00:00-04:00,142.479996,143.580002,140.850006,141.970001,180820600,0.0,0.0
2025-06-16 00:00:00-04:00,143.350006,146.179993,143.199997,144.690002,183133700,0.0,0.0


In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer

dates = pd.to_numeric(nvda_hist.index)
dates = dates.to_numpy()
dates

In [49]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
scaler = StandardScaler()
dates_scaled = scaler.fit_transform(dates.reshape(-1, 1))
print(dates_scaled)

In [ ]:
normalizer = Normalizer()
dates_norm = normalizer.fit_transform(dates.reshape(-1, 1))
print(dates_norm)